In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix

In [3]:
df=pd.read_csv('IMDB Dataset.csv')

In [4]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
df.shape

(50000, 2)

In [6]:
df=df.head(10000)

In [7]:
df.shape

(10000, 2)

In [8]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [9]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [10]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
df.duplicated().sum()

17

In [13]:
df.drop_duplicates(inplace=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_9688\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [14]:
df.duplicated().sum()

0

# basic preprocessing

In [15]:
# remove html tag
import re

def remove_html_tags(text):
    return re.sub(r"</?[^>]+>", "", text)

In [16]:
df['review']=df['review'].apply(remove_html_tags)

C:\Users\dell\AppData\Local\Temp\ipykernel_9688\2177839855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_html_tags)


In [17]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [18]:
# lowercase
import re

def to_lowercase(text):
    return re.sub(r"[A-Z]", lambda m: m.group().lower(), text)

In [19]:
df['review']=df['review'].apply(to_lowercase)

C:\Users\dell\AppData\Local\Temp\ipykernel_9688\1082386297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(to_lowercase)


In [20]:
df['review'][10]

'phil the alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.at first it was very odd and pretty funny but as the movie progressed i didn\'t find the jokes or oddness funny anymore.its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually i just lost interest.i imagine this film would appeal to a stoner who is currently partaking.for something similar but better try "brother from another planet"'

In [21]:
# remove stopwords
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")

def remove_stopwords_nltk(text):
    stop_words = set(stopwords.words("english"))
    words = re.findall(r'\b\w+\b', text)  # Extract words using regex
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
df['review']=df['review'].apply(remove_stopwords_nltk)

C:\Users\dell\AppData\Local\Temp\ipykernel_9688\2753591628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_stopwords_nltk)


In [23]:
df['review'][0]

'one reviewers mentioned watching 1 oz episode hooked right exactly happened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home many aryans muslims gangstas latinos christians italians irish scuffles death stares dodgy dealings shady agreements never far away would say main appeal show due fact goes shows dare forget pretty pictures painted mainstream audiences forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards sold nickel inmates kill order get away well mannered middle class inmates turned prison bitches due lack street ski

In [24]:
x=df.drop('sentiment',axis=1)

In [25]:
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically family little boy jake thinks zombie...
4,petter mattei love time money visually stunnin...
...,...
9995,fun entertaining movie wwii german spy julie a...
9996,give break anyone say good hockey movie know m...
9997,movie bad movie watching endless series bad ho...
9998,movie probably made entertain middle school ea...


In [26]:
y=df['sentiment']

In [27]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [28]:
x.shape

(9983, 1)

In [29]:
y.shape

(9983,)

In [30]:
x.head()

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically family little boy jake thinks zombie...
4,petter mattei love time money visually stunnin...


In [31]:
y.head()

0    positive
1    positive
2    positive
3    negative
4    positive
Name: sentiment, dtype: object

In [32]:
# label encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [33]:
y=encoder.fit_transform(y)

In [34]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [35]:
# train_test_split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [36]:
x_train

,review
6940,back 1993 sega released dull lackluster video ...
1919,10 viewings 20 years think crazy gang best eff...
718,hard praise film much cgi dragon well done lac...
8599,rented horrible movie worst think ever seen be...
7950,spoilers one worst films seen since last years...
...,...
5737,cowboys james stewart walter brennan take herd...
5194,movie look feel put together matter days kind ...
5393,total crap kind excited see film version seen ...
860,production quite surprise absolutely love obsc...


In [37]:
x_test

,review
1254,good grief still maintain manos hands fate wor...
8266,thought intensive care quite bad unintentional...
1662,wonderful film mixes documentary fiction way m...
7472,cool flick enjoyable watch hope see fred carpe...
6537,boy collision deafening female telephone linem...
...,...
9974,though gang comedies still followers got say a...
3524,contains spoilers truly exquisite sean young s...
9686,deceptively laid back low key casually paced a...
2251,opening dialog scenes knew knew train wreck wa...


In [38]:
y_train

array([0, 1, 0, ..., 0, 1, 0])

In [39]:
y_test

array([0, 0, 1, ..., 1, 0, 0])

In [40]:
x_test.shape

(1997, 1)

In [41]:
x_train.shape

(7986, 1)

In [42]:
y_train.shape

(7986,)

In [43]:
y_test.shape

(1997,)

In [44]:
# applying BoW
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()



In [45]:
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()

In [46]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [47]:
x_test_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
# naive_bayes
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [52]:
y_pred=gnb.predict(x_test_bow)

In [53]:
# accuracy measurement
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.629444166249374

In [54]:
confusion_matrix(y_test,y_pred)

array([[697, 288],
       [452, 560]], dtype=int64)

In [72]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)
y_pred=rf.predict(x_test_bow)
accuracy_score(y_test,y_pred)

0.8302453680520782

In [73]:
# applying max_features
cv=CountVectorizer(max_features=3000)
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()

In [74]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [75]:
x_test_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0]], dtype=int64)

In [76]:
rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)
y_pred=rf.predict(x_test_bow)
accuracy_score(y_test,y_pred)

0.8402603905858789

# N Grams

In [77]:
cv=CountVectorizer(ngram_range=(1,2),max_features=3000)
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()

rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)
y_pred=rf.predict(x_test_bow)
accuracy_score(y_test,y_pred)


0.842764146219329

# TFIDF

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [79]:
x_train.shape

(7986, 1)

In [80]:
x_test.shape

(1997, 1)

In [81]:
y_train.shape

(7986,)

In [82]:
y_test

array([0, 0, 1, ..., 1, 0, 0])

In [83]:
x_train_tfidf=tfidf.fit_transform(x_train['review']) # toarray() dont be used
x_test_tfidf=tfidf.transform(x_test['review']) # toarray() dont be used

In [84]:
x_train_tfidf.shape

(7986, 48105)

In [85]:
x_test_tfidf.shape

(1997, 48105)

In [86]:
rf=RandomForestClassifier()
rf.fit(x_train_tfidf,y_train)


RandomForestClassifier()

In [87]:
y_pred=rf.predict(x_test_tfidf)
accuracy_score(y_test,y_pred)

0.8602904356534802

# THE END
# THANK YOU EVERYONE
# MD NAZMUL HAQUE 
